# Source dataset: oxford-iiit-pet

In [2]:
import os


img_list = ['_'.join(i.split('_')[:-1]) for i in os.listdir('/media/disk1/Data/oxford-iiit-pet/images') if i.endswith('.jpg')]
sets = set(img_list)

print('total number of dataset:', len(img_list))

print()
print('--num dataset != 200')

for name in sets:
    if img_list.count(name) != 200:
        print(name, len([i for i in img_list if i == name]))



total number of dataset: 7390

--num dataset != 200
staffordshire_bull_terrier 191
scottish_terrier 199


# SpurioudCatDog dataset generation

In [3]:
from prepare_dataset import PrepareDataset

prep = PrepareDataset(
    places365_root='/media/disk2/Data',
    save_root='/media/disk1/Data',
    oxford_iiit_pet_root='/media/disk1/Data',
    dataset="SpuriousCatDogVer4", 
    download=False)

SpuriousCatDogVer4 dataset is already prepared


# Checking dataset

In [1]:
from data_module import DataModule

dm = DataModule('SpuriousCatDogVer3', '/media/disk1/Data', 1234, 2, 16, 16, None, None, None, None)

/home/jj/anaconda3/envs/torch2.1_cuda11.8/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/jj/anaconda3/envs/torch2.1_cuda11.8/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


ImportError: attempted relative import with no known parent package

# Spurious CatDog generation code debugging 

In [ ]:
import pandas as pd
import os

# df = pd.read_csv('/home/data/waterbirds/waterbird_complete95_forest2water2/metadata.csv')
# df.keys()

In [ ]:
import numpy as np
root = '/media/disk1/Data/oxford-iiit-pet/'
img_dir = os.path.join(root, 'images')
mask_dir = os.path.join(root, 'annotations/trimaps')

img_list = sorted([i for i in os.listdir(img_dir) if '.jpg' in i])

df = pd.DataFrame(img_list, columns=['fg_filename'])

df['species'] = df['fg_filename'].apply(lambda x: '_'.join(x.split('_')[:-1]))
df['y'] = df['species'].apply(lambda x: 0 if x[0].isupper() else 1)

# set 'split' and 'place' of df.
# The correlation between 'place' and 'y' is 0.95 for training dataset
# The correlation between 'place' and 'y' is 0.9 for validation dataset
df['split'] = 'train'
df['place'] = 0
for species in df['species'].unique():
    species_df = df[df['species'] == species]
    test_idx = species_df.sample(frac=0.2, random_state=42).index
    val_idx = species_df.drop(test_idx).sample(frac=0.2, random_state=42).index
    train_idx = species_df.drop(test_idx).drop(val_idx).index
    
    df.loc[test_idx, 'split'] = 'test'
    df.loc[val_idx, 'split'] = 'val'

    # Set 'place'
    label0_idx = df[df['y'] == 0].index
    label1_idx = df[df['y'] == 1].index

    df.loc[label0_idx.join(train_idx, how='inner'), 'place'] = np.random.choice([0, 1], size=len(label0_idx.join(train_idx, how='inner')), p=[0.95, 0.05])
    df.loc[label1_idx.join(train_idx, how='inner'), 'place'] = np.random.choice([0, 1], size=len(label1_idx.join(train_idx, how='inner')), p=[0.05, 0.95])
    df.loc[label0_idx.join(val_idx, how='inner'), 'place'] = np.random.choice([0, 1], size=len(label0_idx.join(val_idx, how='inner')), p=[0.5, 0.5])
    df.loc[label1_idx.join(val_idx, how='inner'), 'place'] = np.random.choice([0, 1], size=len(label1_idx.join(val_idx, how='inner')), p=[0.5, 0.5])

test_idx = df[df['split'] == 'test'].index
df.loc[test_idx, 'place'] = 0
test_df = df.loc[test_idx].copy(deep=True)
test_df['place'] = 1

df = pd.concat([df, test_df], axis=0)
df = df.reset_index()

# allocates place_filename to each row of df


# load yaml file
import yaml
with open('/home/jj/Research/ConceptualSensitivityRegularization/configs/dataset/waterbirds_used_bgs.yaml', 'r') as f:
    config_waterbirds_bg = yaml.load(f, Loader=yaml.FullLoader)

places_dict = {
    0: config_waterbirds_bg['water'],
    1: config_waterbirds_bg['land']
}

df.loc[df['place'] == 0, 'place_filename'] = places_dict[0][:sum(df['place'] == 0)]
df.loc[df['place'] == 1, 'place_filename'] = places_dict[1][:sum(df['place'] == 1)]

df

In [ ]:
df.groupby(['species', 'split', 'y', 'place']).count().pivot_table(index=['split', 'y', 'place'], columns=['species'], values='fg_filename', aggfunc='sum')


In [ ]:
df.groupby(['split', 'y', 'place']).count()

In [ ]:
df.groupby(['split']).count()

In [ ]:
def crop_and_resize(source_img, target_img):
    """
    Make source_img exactly the same as target_img by expanding/shrinking and
    cropping appropriately.

    If source_img's dimensions are strictly greater than or equal to the
    corresponding target img dimensions, we crop left/right or top/bottom
    depending on aspect ratio, then shrink down.

    If any of source img's dimensions are smaller than target img's dimensions,
    we expand the source img and then crop accordingly

    Modified from
    https://stackoverflow.com/questions/4744372/reducing-the-width-height-of-an-image-to-fit-a-given-aspect-ratio-how-python
    """
    source_width = source_img.size[0]
    source_height = source_img.size[1]

    target_width = target_img.size[0]
    target_height = target_img.size[1]

    # Check if source does not completely cover target
    if (source_width < target_width) or (source_height < target_height):
        # Try matching width
        width_resize = (target_width, int((target_width / source_width) * source_height))
        if (width_resize[0] >= target_width) and (width_resize[1] >= target_height):
            source_resized = source_img.resize(width_resize, Image.ANTIALIAS)
        else:
            height_resize = (int((target_height / source_height) * source_width), target_height)
            assert (height_resize[0] >= target_width) and (height_resize[1] >= target_height)
            source_resized = source_img.resize(height_resize, Image.ANTIALIAS)
        # Rerun the cropping
        return crop_and_resize(source_resized, target_img)

    source_aspect = source_width / source_height
    target_aspect = target_width / target_height

    if source_aspect > target_aspect:
        # Crop left/right
        new_source_width = int(target_aspect * source_height)
        offset = (source_width - new_source_width) // 2
        resize = (offset, 0, source_width - offset, source_height)
    else:
        # Crop top/bottom
        new_source_height = int(source_width / target_aspect)
        offset = (source_height - new_source_height) // 2
        resize = (0, offset, source_width, source_height - offset)

    source_resized = source_img.crop(resize).resize((target_width, target_height), Image.ANTIALIAS)
    return source_resized


def combine_and_mask(img_new, mask, img_black):
    """
    Combine img_new, mask, and image_black based on the mask

    img_new: new (unmasked image)
    mask: binary mask of bird image
    img_black: already-masked bird image (bird only)
    """
    # Warp new img to match black img
    img_resized = crop_and_resize(img_new, img_black)
    img_resized_np = np.asarray(img_resized)

    # Mask new img
    img_masked_np = np.around(img_resized_np * (1 - mask)).astype(np.uint8)

    # Combine
    img_combined_np = np.asarray(img_black) + img_masked_np
    img_combined = Image.fromarray(img_combined_np)

    return img_combined

In [ ]:
def resize_and_pad(img, ratio):
    """
    Resize img to ratio and pad with black at center of the original image
    """
    width = img.size[0]
    height = img.size[1]
    img_resized = img.resize((int(width * ratio), int(height * ratio)), Image.ANTIALIAS)

    # Pad with black
    img_padded = Image.new('RGB', (width, height), (0, 0, 0))
    img_padded.paste(img_resized, ((width - img_resized.size[0]) // 2, (height - img_resized.size[1]) // 2))

    return img_padded

In [ ]:
from PIL import Image

root = '/media/disk1/Data/'
fg_dir = os.path.join(root, 'oxford-iiit-pet/images')
mask_dir = os.path.join(root, 'oxford-iiit-pet/annotations/trimaps')
bg_dir = os.path.join(root, 'Places205/data/vision/torralba/deeplearning/images256/')

for i, row in df.iterrows():
    fg_fname = row['fg_filename']
    bg_fname = row['place_filename']

    fg_path = os.path.join(fg_dir, fg_fname)
    bg_path = os.path.join(bg_dir, bg_fname)
    mask_path = os.path.join(mask_dir, fg_fname.replace('.jpg', '.png'))

    fg = Image.open(fg_path).convert('RGB')
    mask = Image.open(mask_path).convert('RGB')
    bg = Image.open(bg_path).convert('RGB')

    fg_np =  np.asarray(resize_and_pad(fg, 0.7))
    mask_np = np.float32(np.asarray(resize_and_pad(mask, 0.7)) == 1)

    fg_only = Image.fromarray(np.around(fg_np * mask_np).astype(np.uint8))
    combined_img = combine_and_mask(bg, mask_np, fg_only)


    break

In [ ]:
def resize_and_pad(img, ratio):
    """
    Resize img to ratio and pad with black at center of the original image
    """
    width = img.size[0]
    height = img.size[1]
    img_resized = img.resize((int(width * ratio), int(height * ratio)), Image.ANTIALIAS)

    # Pad with black
    img_padded = Image.new('RGB', (width, height), (0, 0, 0))
    img_padded.paste(img_resized, ((width - img_resized.size[0]) // 2, (height - img_resized.size[1]) // 2))

    return img_padded

resize_and_pad(fg, 0.7)

In [ ]:
save_filename = f"{row['fg_filename'].split('.')[0]}_{row['place_filename'].split('/')[1]}"

In [ ]:

dataset_save_path = os.path.join(root, '/home/data/SpuriousCatDogVer3')
os.path.join(root, "SpuriousCatDogVer3", dataset_save_path)
